## Calculate summary statistic (min, max, mean, count,...) per group

In [7]:
import numpy as np
import pandas as pd


In [8]:
local_relative_path = "./source/data_processed/wellbore_exploration_all_clean_names.csv"
wellbore_exploration_all = pd.read_csv(local_relative_path)
wellbore_exploration_all.head()

,wellbore_name,well,drilling_operator,production_licence,purpose,status,content,well_type,sub_sea,entry_date,...,npdid_wellbore,dsc_npdid_discovery,npdid_field,npdid_facility_drilling,npdid_wellbore_reclass,l_npdid_production_licence,npdid_site_survey,date_updated,date_updated_max,datesync_npd
0,1/2-1,1/2-1,Phillips Petroleum Norsk AS,143,WILDCAT,P&A,OIL,EXPLORATION,NO,20.03.1989,...,1382,43814.0,3437650.0,296245.0,0,21956.0,NaN,03.10.2019,03.10.2019,22.11.2019
1,1/2-2,1/2-2,Paladin Resources Norge AS,143 CS,WILDCAT,P&A,OIL SHOWS,EXPLORATION,NO,14.12.2005,...,5192,NaN,NaN,278245.0,0,2424919.0,NaN,03.10.2019,03.10.2019,22.11.2019
2,1/3-1,1/3-1,A/S Norske Shell,011,WILDCAT,P&A,GAS,EXPLORATION,NO,06.07.1968,...,154,43820.0,NaN,288604.0,0,20844.0,NaN,03.10.2019,03.10.2019,22.11.2019
3,1/3-2,1/3-2,A/S Norske Shell,011,WILDCAT,P&A,DRY,EXPLORATION,NO,14.05.1969,...,165,NaN,NaN,288847.0,0,20844.0,NaN,03.10.2019,03.10.2019,22.11.2019
4,1/3-3,1/3-3,Elf Petroleum Norge AS,065,WILDCAT,P&A,OIL,EXPLORATION,NO,22.08.1982,...,87,43826.0,1028599.0,288334.0,0,21316.0,NaN,03.10.2019,03.10.2019,22.11.2019


What is the minimum and maximum total_depth grouped by completion_year and sorted by entry_year?

We see that well_bore name was silently dropped since it is a categorical column.

In [9]:
(wellbore_exploration_all
  .filter(items=["wellbore_name", "entry_year", "total_depth"])
  .groupby("entry_year")
  .agg(
    total_depth_min = pd.NamedAgg(column="total_depth", aggfunc="min"),
    total_depth_max = pd.NamedAgg(column="total_depth", aggfunc="max")
    )
  .sort_values("entry_year", ascending = False)
  .reset_index()
  ).head(10)

,entry_year,total_depth_min,total_depth_max
0,2019,0.0,6068.0
1,2018,381.0,6100.0
2,2017,508.0,7811.0
3,2016,488.0,7126.0
4,2015,840.0,5765.0
5,2014,1033.0,4889.0
6,2013,930.0,5634.0
7,2012,1640.0,5573.0
8,2011,1339.0,5672.0
9,2010,1178.0,5738.0


## Add a new calculated column

### What is lambda or anonymus function?

In [10]:
add_ten = lambda x: x + 1

In [11]:
add_ten(10)

11

### New calculated column

assign method returns a new object with all original columns in addition to new ones. Existing columns that are re-assigned will be overwritten. For adding new columns to a DataFrame in a chain (inspired by dplyr's mutate)

depth_flag will be the new column that will have two values, if total_depth is larger or equal to 2000m, it will be classified as "deep", otherwise it will be classified as "shallow".

In [12]:
(wellbore_exploration_all
    .filter(items=["wellbore_name", "total_depth", "utm_zone"])
    .assign(
        depth_flag=lambda x: np.where(x["total_depth"] >= 1000, "deep", "shallow")
        )  
)

,wellbore_name,total_depth,utm_zone,depth_flag
0,1/2-1,3574.0,31,deep
1,1/2-2,3434.0,31,deep
2,1/3-1,4877.0,31,deep
3,1/3-2,4297.0,31,deep
4,1/3-3,4876.0,31,deep
...,...,...,...,...
1917,7324/10-1,2919.0,35,deep
1918,7325/1-1,2865.0,35,deep
1919,7325/4-1,1210.0,35,deep
1920,7335/3-1,4300.0,37,deep


### Add calculated columns that are dependent on each othe

In Python 3.6+, you can create multiple columns within the same assign where one of the columns depends on another one defined within the same assign.

We will calculate two new columns:

* `drilling_depth_flag` column which is when `total_depth` column has values larger or equal to 1000m, where they will be classified as "deep", and otherwise as "shallow"
* `depth_age_flag` column is dependent on the previously calculated column `drilling_depth_flag`. Output of the column is "deep_and_old" if "drilling_depth_flag" column is "deep") AND if "completion_year" column is less and equal than 2010, otherwise it is "shallow_and_new".

In [14]:
(wellbore_exploration_all
    .filter(items=["wellbore_name", "total_depth", "utm_zone", "completion_year"])
    .assign(
        drilling_depth_flag=lambda x: np.where(x["total_depth"] >= 1000, "deep", "shallow"),
        depth_age_flag=lambda x: np.where(
            (x["drilling_depth_flag"] == "deep") & (x["completion_year"] <= 2010),   # drilling_depth_flag has been created in previous step 
            "deep_and_old", "deep_and_new")
        )  
)

,wellbore_name,total_depth,utm_zone,completion_year,drilling_depth_flag,depth_age_flag
0,1/2-1,3574.0,31,1989,deep,deep_and_old
1,1/2-2,3434.0,31,2006,deep,deep_and_old
2,1/3-1,4877.0,31,1968,deep,deep_and_old
3,1/3-2,4297.0,31,1969,deep,deep_and_old
4,1/3-3,4876.0,31,1983,deep,deep_and_old
...,...,...,...,...,...,...
1917,7324/10-1,2919.0,35,1989,deep,deep_and_old
1918,7325/1-1,2865.0,35,2014,deep,deep_and_new
1919,7325/4-1,1210.0,35,2017,deep,deep_and_new
1920,7335/3-1,4300.0,37,2019,deep,deep_and_new


### Calculated column - define multiple cases with np.select

In [15]:
df_multiple = (wellbore_exploration_all
    .filter(items=["wellbore_name", "total_depth", "water_depth"])
    .assign(
        water_depth_category = lambda x: np.select(
            condlist = [
                x["water_depth"].between(0, 305, inclusive=True),
                x["water_depth"].between(306, 1524, inclusive=True),
                x["water_depth"].between(1525, 2133, inclusive=True),
                x["water_depth"].between(2134, np.max(x["water_depth"]), inclusive=True)
                ],
            choicelist = [ 
                "Shallow Water",
                "Midwater",
                "Deepwater",
                "Ultra-Deepwater"
            ],
            default="Unknown" 
        )
    )
)
df_multiple["water_depth_category"].value_counts()    
#df_multiple.query("water_depth_category == 'Ultra Deep'")

Shallow Water    1373
Midwater          547
Deepwater           2
Name: water_depth_category, dtype: int64

In [16]:
df_multiple.sample(10)

,wellbore_name,total_depth,water_depth,water_depth_category
1437,6406/2-7,4981.0,293.0,Shallow Water
1331,35/10-4 A,3946.0,363.0,Midwater
1445,6406/3-6,4175.0,273.0,Shallow Water
695,25/7-1 S,3592.0,127.0,Shallow Water
1364,35/12-1,3020.0,351.0,Midwater
538,16/7-8 S,2900.0,79.5,Shallow Water
1055,33/9-11,3528.0,287.0,Shallow Water
605,24/12-3 S,3058.0,118.0,Shallow Water
335,15/5-4,2300.0,120.0,Shallow Water
1256,34/10-45 A,6523.0,142.5,Shallow Water


### Converting columns to date and time, and calculating difference

In [28]:
wellbore_exploration_all.columns

df_days = (wellbore_exploration_all
    .filter(items = ['well', 'entry_date','completion_date'])
    .assign(
        entry_date=pd.to_datetime(wellbore_exploration_all['entry_date']),
        completion_date=pd.to_datetime(wellbore_exploration_all['completion_date']),
        days_of_drilling = lambda x: (x["completion_date"] - x["entry_date"]).dt.days,
        )
    )
df_days.head()

,well,entry_date,completion_date,days_of_drilling
0,1/2-1,1989-03-20,1989-04-06,17.0
1,1/2-2,2005-12-14,2006-02-02,50.0
2,1/3-1,1968-06-07,1968-11-11,157.0
3,1/3-2,1969-05-14,1969-07-27,74.0
4,1/3-3,1982-08-22,1983-03-24,214.0
